# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
import pandas as pd
url="https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
customers=pd.read_csv(url)
customers.head()


In [ ]:
#1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.
condition1= (customers["Response"]=="Yes") & (customers["Total Claim Amount"]>1000)
customer_condition1=customers[condition1]
customer_condition1

In [ ]:
#2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.
condition2=customers["Response"]=="Yes"
customers_condition2=customers[condition2]
customers_condition2.groupby(["Policy Type", "Gender"]).agg({"Total Claim Amount":["mean"]})

In [ ]:
#3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.
customers_state=customers.groupby("State").agg({"Customer":"count"})
condition2=customers_state["Customer"]> 500
customers_state2=customers_state[condition2]
customers_state2


In [ ]:
#4.Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.
customers_lifetime=customers.groupby(["Education", "Gender"]).agg({"Customer Lifetime Value": ["median","max","min"]})
customers_lifetime

In [ ]:
#5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.
customers['Effective To Date'] = pd.to_datetime(customers['Effective To Date'])
customers.dtypes
customers["Month"]=customers["Effective To Date"].dt.month
pivot=customers.pivot_table(index="Month",columns="State", values="Customer", aggfunc="count")
pivot

In [ ]:
#6.Filter the top_state
customers["Number of Policies"]=customers["Number of Policies"].astype(int)
top_state=((pd.DataFrame(customers.groupby("State")["Number of Policies"].sum())).sort_values("Number of Policies",ascending=False)).head(3)
top_state=top_state.reset_index()
top_state_list=top_state["State"].tolist()
top_state_list

In [ ]:
#6. Create the table with the top_state
condition6=customers["State"].isin(top_state_list)
customers6=customers[condition6]
customers6=customers6.groupby(["State", "Month"]).agg({"Customer":"count"}).sort_values(by="Customer", ascending=False)
customers6

In [ ]:
#7.The marketing team wants to analyze the effect of different marketing channels on the customer response rate.
#Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

response_rate = customers.groupby('Sales Channel')['Response'].value_counts(normalize=True).unstack()
response_rate